First gen data 

In [1]:
import numpy as np
import statsmodels.formula.api as sm
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn import tree
import scipy.interpolate

/Users/andrevauvelle/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [13]:
ls

Estimating heterogeneous treatment effects using random forests.ipynb
a.txt
data.csv
projectpitches.docx
sort.py
venv/


In [161]:
df = pd.read_csv('data.csv')

/Users/andrevauvelle/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [162]:
df.head()

,transfers.subject_id,transfers.hadm_id,transfers.icustay_id,transfers.eventtype,team_census,blocked_count,remaining_beds,icustay_los_total,icustay_los_boarder,icustay_boarder_initial,...,sofa_respiration,sofa_coagulation,sofa_liver,sofa_cardiovascular,sofa_cns,sofa_renal,days_survived_dt,days_survived_dt2,time_diff,icustay_outtime_deathtime_difference
0,18,188822,298129,admit,14,4,0,30.92,30.92,1,...,NaN,0.0,0.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN
1,33,176176,296681,admit,11,5,1,37.30,0.00,0,...,NaN,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
2,52,190797,261857,admit,17,0,1,44.41,0.00,0,...,NaN,3.0,3.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
3,59,104130,224440,admit,15,0,4,52.84,0.00,0,...,NaN,0.0,0.0,1.0,0.0,4.0,NaN,NaN,NaN,NaN
4,62,116009,216609,admit,11,6,0,67.82,67.82,1,...,NaN,1.0,NaN,3.0,0.0,0.0,NaN,NaN,NaN,NaN


In [163]:
#columns = ('congestive_heart_failure', 'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation', 'peripheral_vascular', 'hypertension')
columns = ('remaining_beds', 'icustay_admit_age', 'gender', 'oasis', 'elixhauser_28day', 'team_census')

# No. dimensions of covariates
d = len(columns)

X = df.loc[:,columns].reset_index(drop=True)

tau = df['days_survived']

# Remove nans
drop1 = np.isnan(tauxy.values)
drop2 = np.isnan(X.values).any(axis=1)
drop = drop1 | drop2
tauxy = tauxy[(~drop)].reset_index(drop=True)
X = X[~drop].reset_index(drop=True)

ValueError: operands could not be broadcast together with shapes (8437,) (8442,) 

In [160]:
# Draw a random subsample of size s.
n = X.shape[0]
s = n/2
# Choose s ints between 0 and n randomly
subSampleMask = random.sample(range(0, n), int(s))
# Create set I
setIMask = random.sample(subSampleMask, np.int(np.ceil(s/2.)))
setI = [X.iloc[setIMask]]
dfSetI = pd.DataFrame(data=X.iloc[setIMask])
# Create set J
setJMask = [i for i in subSampleMask if i not in setIMask]
setJ = [X.iloc[setJMask]]

In [156]:
# Create tree on the J set
clf = tree.DecisionTreeRegressor(criterion='mse')
clf = clf.fit(setJ[0], tau[setJMask])

In [157]:
clf

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [158]:
tau_hat = clf.predict(dfSetI.iloc[:,:d])
# Out of sample R^2:
clf.score(dfSetI.iloc[:,:d],tau[setIMask])

-0.0033554555896646576

In [147]:
def plotResults(dfSetI,tau_hat):
    # Set up a regular grid of interpolation points
    xi, yi = np.linspace(dfSetI.iloc[:,0].min(),
                         dfSetI.iloc[:,0].max(), 100), np.linspace(dfSetI.iloc[:,1].min(), dfSetI.iloc[:,1].max(), 100)
    xi, yi = np.meshgrid(xi, yi)
    # Interpolate
    rbf = scipy.interpolate.Rbf(dfSetI.iloc[:,0], dfSetI.iloc[:,1], tau_hat, function='linear')
    zi = rbf(xi, yi)

    plt.imshow(zi, vmin=1., vmax=4., origin='lower',
               extent=[dfSetI.iloc[:,0].min(), dfSetI.iloc[:,0].max(), dfSetI.iloc[:,1].min(), dfSetI.iloc[:,1].max()],
              cmap='plasma')
    plt.colorbar()
    plt.show()